# Kiva Project: Computer Science

Foundation of Computer Science Project on Kiva Dataset

Mattia Rigiroli, 819987

21-01-2020

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Importo tutte le librerie necessarie:

In [0]:
import pandas as pd
import numpy as np
import time
import datetime
import random

# 1- Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [0]:
loan_lenders=pd.read_csv("/content/drive/My Drive/CS Project/Kiva_Dataset/loans_lenders.csv")

In [4]:
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [5]:
loan_short=loan_lenders[0:100]
df = pd.DataFrame({'loan_id':[], 'lender':[]})
for i in range(0,len(loan_short)):
    loan_id=loan_short["loan_id"][i]
    lenders=loan_short['lenders'][i]
    lenders_list=lenders.split(',')
    for k in range(0,len(lenders_list)):
        df = pd.DataFrame(np.array([[loan_id,lenders_list[k]]]),columns=["loan_id", "lender"]).append(df, ignore_index=True)
df.head()

,loan_id,lender
0,338792,emily3379
1,338792,shannon3680
2,338792,anders5148
3,338792,scott9318
4,282917,rs7257


Questo metodo applicato all'intero dataset risulta molto lento e inefficace. Ho quindi applicato un altra metodologia.

Elaborazione sull'intero dataset:

In [6]:
# Creo una lista di ID 
id_list=[x for x in loan_lenders['loan_id']]
id_list[0:10]

[483693,
 483738,
 485000,
 486087,
 534428,
 558112,
 563395,
 575414,
 578029,
 551251]

In [7]:
# Creo una lista di lenders
lenders_list=[y for x in loan_lenders['lenders'] for y in x.split(',')]
lenders_list[0:20]

['muc888',
 ' sam4326',
 ' camaran3922',
 ' lachheb1865',
 ' rebecca3499',
 ' karlheinz4543',
 ' jerrydb',
 ' paula8951',
 ' gmct',
 ' amra9383',
 ' r3922',
 ' brian9451',
 ' shree8053',
 ' alan5513',
 ' oisin3389',
 ' helle8622',
 ' bo3186',
 ' ric8947',
 ' daniel98469874',
 ' nick9464']

In [8]:
len(lenders_list)

28293931

In [9]:
# Creo una lista che contiene il numero di lenders per ciascun ID
# In questo caso, al primo loan_id corrispondono 40 lenders e così via. 
lunghezza=[len(x.split(',')) for x in loan_lenders['lenders']]
lunghezza[0:10]

[40, 15, 15, 13, 19, 13, 11, 23, 13, 32]

In [10]:
# Aggiungo ad una nuova lista ciascun ID presente nella lista id_list per il numero di volte 
# presente in lunghezza (n di lenders per ciascun ID). 
id_list1=[]
for i in range(0,len(lunghezza)):
    id_list1.append([id_list[i]]*lunghezza[i])
# Il risultato è una lista di liste: estraggo perciò i singoli ID e li metto in un'unica lista. 
id_finale=[y for x in id_list1 for y in x]
id_finale[0:10]    

[483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693,
 483693]

In [11]:
# Verifico che il numero di ID sia uguale al numero di lenders. 
len(id_finale)

28293931

In [0]:
loan_lender_norm = pd.DataFrame({'loan_id':id_finale, 'lender':lenders_list})

In [13]:
loan_lender_norm

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# 2- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [0]:
loans=pd.read_csv("/content/drive/My Drive/CS Project/Kiva_Dataset/loans.csv")

In [15]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [16]:
type(loans['disburse_time'][1])

str

In [17]:
type(loans['planned_expiration_time'][1])

str

In [18]:
loans['disburse_time'][1]

'2013-12-20 08:00:00.000 +0000'

In [0]:
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['disburse_time']= pd.to_datetime(loans['disburse_time'])

In [0]:
loans['loans_duration']=abs(loans['planned_expiration_time']-loans['disburse_time'])

In [21]:
loans['loans_duration'][0:5]

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: loans_duration, dtype: timedelta64[ns]

Ho effettuato un controllo sui valori mancanti e quando disburse_time o planned_expiration_time sono nulle (o entrambe sono nulle), loans_duration è nulla.

In [22]:
loans[['loan_id','loans_duration']].head(10)

,loan_id,loans_duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
5,659605,35 days 18:30:07
6,660240,31 days 23:50:11
7,661601,46 days 01:50:03
8,661960,33 days 16:10:03
9,662117,68 days 05:40:02


In [23]:
loans[['loan_id','loans_duration']].tail(10)

,loan_id,loans_duration
1419597,984735,63 days 06:20:02
1419598,985764,46 days 02:40:05
1419599,986756,44 days 12:20:04
1419600,986789,40 days 11:50:04
1419601,987606,42 days 08:30:11
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02
1419606,989240,61 days 12:50:06


# 3- Find the lenders that have funded at least twice.

In [24]:
loan_lender_norm.head()

,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [0]:
d_funded=loan_lender_norm.groupby('lender').count()
d_funded.rename(columns={'loan_id': 'n loan x lender'}, inplace=True)

In [26]:
d_funded.head()

,n loan x lender
lender,
000,39
00000,39
0002,70
00mike00,1
0101craign0101,71


In [27]:
# Seleziono solamente i lenders che hanno finanziato almeno due volte e creo un nuovo DataFrame. 
d_funded_2=d_funded[d_funded['n loan x lender']>=2].sort_values(by='n loan x lender', ascending=False)
d_funded_2

,n loan x lender
lender,
gooddogg1,151307
trolltech4460,148347
gmct,127089
nms,100360
themissionbeltco,76986
...,...
omurhu9493,2
beth36573246,2
omri1441,2


# 4-  For each country, compute how many loans have involved that country as borrowers.

In [28]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [29]:
count_borrowers=pd.DataFrame(loans.groupby('country_name').count()['loan_id'])
count_borrowers.rename(columns={'loan_id': 'tot_loan'}, inplace=True)
count_borrowers=count_borrowers.sort_values(by='tot_loan', ascending=False)
count_borrowers

,tot_loan
country_name,
Philippines,285336
Kenya,143699
Peru,86000
Cambodia,79701
El Salvador,64037
...,...
Canada,1
Papua New Guinea,1
Botswana,1


# 5-For each country, compute the overall amount of money borrowed.

In [30]:
cash_borrowed=pd.DataFrame(loans.groupby('country_name').sum()['loan_amount'])
cash_borrowed.rename(columns={'loan_amount': 'overall_amount'}, inplace=True)
cash_borrowed=cash_borrowed.sort_values(by='overall_amount', ascending=False)
cash_borrowed

,overall_amount
country_name,
Philippines,97984600.0
Peru,79437775.0
Kenya,66735975.0
Paraguay,53964700.0
Cambodia,51613525.0
...,...
Virgin Islands,10000.0
Vanuatu,9250.0
Botswana,8000.0


In [31]:
# Calcolo il totale di soldi finanziati che servirà nei punti successivi. 
totale=sum(cash_borrowed['overall_amount'])
totale

1181437300.0

# 6-Like the previous point, but expressed as a percentage of the overall amount lent.


In [32]:
perc_cash_borrowed=pd.DataFrame(((loans.groupby('country_name').sum()['loan_amount'])/totale)*100)
perc_cash_borrowed.rename(columns={'loan_amount': 'perc_total_amount'}, inplace=True)
perc_cash_borrowed=perc_cash_borrowed.sort_values(by='perc_total_amount', ascending=False)
perc_cash_borrowed

,perc_total_amount
country_name,
Philippines,8.293678
Peru,6.723825
Kenya,5.648711
Paraguay,4.567716
Cambodia,4.368706
...,...
Virgin Islands,0.000846
Vanuatu,0.000783
Botswana,0.000677


# 7-Like the three previous points, but split for each year (with respect to disburse time).


In [0]:
# Creo una variabile anno per ciascun loans:
loans['disburse_years']=pd.to_datetime(loans['disburse_time']).dt.year

In [34]:
# Calcolo il totale per ogni anno:
tot_year=pd.DataFrame(loans.groupby('disburse_years').sum()['loan_amount'])
tot_year.rename(columns={'loan_amount': 'year_overall_amount'}, inplace=True)
tot_year

,year_overall_amount
disburse_years,
2005.0,102850.0
2006.0,1376575.0
2007.0,15446525.0
2008.0,39423050.0
2009.0,59689475.0
2010.0,72609150.0
2011.0,93699300.0
2012.0,119977575.0
2013.0,132043925.0


In [35]:
sum(tot_year['year_overall_amount'])

1170496550.0

In [36]:
perc_cash_borrowed_year=pd.DataFrame(((loans.groupby(['country_name','disburse_years']).sum()['loan_amount'])/(loans.groupby('disburse_years').sum()['loan_amount']))*100)
perc_cash_borrowed_year.rename(columns={'loan_amount': 'perc_total_amount'}, inplace=True)
perc_cash_borrowed_year

perc_total_amount
country_name disburse_years                   
Afghanistan  2007.0                   1.262258
             2008.0                   0.926806
             2009.0                   0.980282
             2010.0                   0.775866
             2011.0                   0.261608
...                                        ...
Zimbabwe     2013.0                   0.513863
             2014.0                   0.861346
             2015.0                   0.464351
             2016.0                   0.489879
             2017.0                   0.745301

[748 rows x 1 columns]

# 8- For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [37]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration,disburse_years
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06,2013.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07,2013.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05,2014.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02,2014.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02,2013.0


In [0]:
loans_sel=loans[['loan_id','num_lenders_total','funded_amount']]

In [0]:
df=pd.merge(loan_lender_norm,loans_sel,on='loan_id')

In [40]:
df.head()

,loan_id,lender,num_lenders_total,funded_amount
0,483693,muc888,44,1225.0
1,483693,sam4326,44,1225.0
2,483693,camaran3922,44,1225.0
3,483693,lachheb1865,44,1225.0
4,483693,rebecca3499,44,1225.0


In [0]:
df['$lenders']=df['funded_amount']/df['num_lenders_total']

In [42]:
df.head()

,loan_id,lender,num_lenders_total,funded_amount,$lenders
0,483693,muc888,44,1225.0,27.840909
1,483693,sam4326,44,1225.0,27.840909
2,483693,camaran3922,44,1225.0,27.840909
3,483693,lachheb1865,44,1225.0,27.840909
4,483693,rebecca3499,44,1225.0,27.840909


In [43]:
df1=pd.DataFrame(df.groupby('lender').sum()['$lenders'])
#df1=df1.sort_values(by='$lenders', ascending=False)
df1=df1.reset_index()
df1

,lender,$lenders
0,000,1485.309656
1,00000,1249.127691
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932
...,...,...
1639021,zzanita,62.500000
1639022,zzcyna7269,44.285714
1639023,zzinnia,32.758621
1639024,zzmcfate,1889.702578


# 9-For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Per ciascun paese, calcolare la differenza tra l'importo complessivo del denaro prestato e l'importo complessivo del denaro preso in prestito. Poiché il paese del prestatore è spesso sconosciuto, si può presumere che la vera distribuzione tra i paesi sia la stessa calcolata dalle file in cui il paese è noto.

In [0]:
lender=pd.read_csv("/content/drive/My Drive/CS Project/Kiva_Dataset/lenders.csv")

In [45]:
lender.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


Imputo i valori mancati per country_code simulando la distribuzione dei valori di country_code non mancanti. 

In [0]:
# Creo un dataset con tutti i valori nulli per country_code:
lender_vuoto=lender[lender['country_code'].isnull()==True]

In [47]:
len(lender_vuoto)

1458635

In [0]:
# Creo un dataset con tutti i valori non nulli per country_code:
lender_pieno=lender[lender['country_code'].isnull()==False]

In [49]:
# Conto quante volte appare ciascun stato nel dataset relativo ai valori non nulli:
n_stati=pd.DataFrame(lender_pieno.groupby('country_code').size())
n_stati.rename(columns={0: 'n_stati'}, inplace=True)
n_stati=n_stati.reset_index()
n_stati.head()

,country_code,n_stati
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4


In [0]:
# Creo una lista in cui ogni country_code è ripetuto per il numero di volte che 
# compare nel dataset dei valori non nulli. 
lista_stati=[x for x in n_stati['country_code']]
volte=[x for x in n_stati['n_stati']]
ll=[]
for i in range(0,len(n_stati)):
    ll.append([lista_stati[i]]*volte[i])
lista_stati_real=[y for x in ll for y in x]

In [51]:
# Vado a riempire la colonna country_code nel dataset vuoto simulando in modo
# casuale (basandomi sulla lista appena creata) il country_code. 
random.seed(123)
ll_casual=[]
for i in range (0,len(lender_vuoto)):
    ll_casual.append(random.sample(lista_stati_real,1)[0])
lender_vuoto['country_code']=ll_casual

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [52]:
len(ll_casual)

1458635

In [0]:
# Unisco i due dataset:
lender_pieno=lender_pieno.append(lender_vuoto)

In [54]:
lender_pieno.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0
39,qian1385,Qian,NaN,NaN,US,1461302036,NaN,NaN,2.0,NaN,0
42,xigg8769,Darla,Madison,NaN,US,1461302503,NaN,NaN,NaN,NaN,0


In [55]:
lender_sel=lender_pieno[['permanent_name','country_code']]
lender_sel.rename(columns={'permanent_name': 'lender'}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [56]:
lender_sel.head()

,lender,country_code
16,naresh2074,US
31,christina27976796,US
37,vikas1098,IN
39,qian1385,US
42,xigg8769,US


In [57]:
df1.head()

,lender,$lenders
0,000,1485.309656
1,00000,1249.127691
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932


In [0]:
df2=pd.merge(df1,lender_sel,on='lender')

In [59]:
df2.head()

,lender,$lenders,country_code
0,000,26.785714,US
1,0161130,25.000000,US
2,0169713,25.000000,US
3,0197462,25.000000,NL
4,0206338,29.687500,US


In [60]:
df3=pd.DataFrame(df2.groupby('country_code').sum()['$lenders'])
#df3=df3.reset_index()
df3

,$lenders
country_code,
AD,101.190476
AE,116712.184488
AF,5305.147977
AG,3130.000000
AL,2321.284472
...,...
XK,471.050824
YE,9779.961032
ZA,29539.443764


In [61]:
cash_borrowed=pd.DataFrame(loans.groupby('country_code').sum()['loan_amount'])
cash_borrowed.rename(columns={'loan_amount': 'overall_amount'}, inplace=True)
#cash_borrowed=cash_borrowed.reset_index()
cash_borrowed

,overall_amount
country_code,
AF,1967950.0
AL,4307350.0
AM,22950475.0
AZ,14784625.0
BA,477250.0
...,...
XK,3083025.0
YE,3444000.0
ZA,1006525.0


In [63]:
difference=pd.DataFrame(df3['$lenders']-cash_borrowed['overall_amount'])
difference.rename(columns={0: 'difference_amount'}, inplace=True)
difference=difference.reset_index()
difference.rename(columns={'index': 'country_code'}, inplace=True)
difference

,country_code,difference_amount
0,AD,NaN
1,AE,NaN
2,AF,-1.962645e+06
3,AG,NaN
4,AL,-4.305029e+06
...,...,...
211,XK,-3.082554e+06
212,YE,-3.434220e+06
213,ZA,-9.769856e+05
214,ZM,-1.978053e+06


In [0]:
# Creo due dizionari che contengono i precedenti dataset:
diz_lenders=df3.to_dict()
diz_lenders=diz_lenders['$lenders']
diz_borrowers=cash_borrowed.to_dict()
diz_borrowers=diz_borrowers['overall_amount']

In [65]:
for i in range(0,len(difference)):
  if pd.isna(difference.iloc[i]['difference_amount'])==True:
    if difference['country_code'][i] in diz_lenders:
      difference['difference_amount'][i]=diz_lenders[difference['country_code'][i]]
    else:
      difference['difference_amount'][i]=diz_borrowers[difference['country_code'][i]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
difference

,country_code,difference_amount
0,AD,1.011905e+02
1,AE,1.167122e+05
2,AF,-1.962645e+06
3,AG,3.130000e+03
4,AL,-4.305029e+06
...,...,...
211,XK,-3.082554e+06
212,YE,-3.434220e+06
213,ZA,-9.769856e+05
214,ZM,-1.978053e+06


# 10- Which country has the highest ratio between the difference computed at the previous point and the population?


In [0]:
country=pd.read_csv('/content/drive/My Drive/CS Project/Kiva_Dataset/country_stats.csv')

In [68]:
country

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [0]:
ratio=pd.merge(country,difference,on='country_code')[['country_code','difference_amount','population','population_below_poverty_line']]

In [70]:
ratio.head()

,country_code,difference_amount,population,population_below_poverty_line
0,IN,-8.882124e+06,1339180127,21.9
1,NG,-7.742670e+06,190886311,70.0
2,MX,-2.503422e+07,129163276,46.2
3,PK,-2.498323e+07,197015955,29.5
4,BD,3.377809e+03,164669751,31.5


In [71]:
ratio['ratio1']=ratio['difference_amount']/ratio['population']
ratio=ratio.sort_values(by='ratio1', ascending=False)
ratio[['country_code','ratio1']]

,country_code,ratio1
163,WS,62.167456
152,LU,0.229620
71,CA,0.156880
155,NO,0.143104
144,AU,0.131057
...,...,...
85,NI,-4.849176
115,MN,-4.988629
81,SV,-6.536577
107,AM,-7.831483


Il Lussemburgo è il paese con il rapporto maggiore.

# 11- Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [72]:
ratio['ratio2']=ratio['difference_amount']/(100-ratio['population_below_poverty_line'])
ratio=ratio.sort_values(by='ratio2', ascending=False)
ratio[['country_code','ratio2']]

,country_code,ratio2
71,CA,63417.212157
36,GB,30002.153784
23,DE,22233.884258
41,FR,21429.974889
94,NL,15966.885776
...,...,...
154,KP,NaN
162,SO,NaN
157,PR,NaN
161,SB,NaN


# 12-For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [73]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,loans_duration,disburse_years
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06,2013.0
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07,2013.0
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05,2014.0
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02,2014.0
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02,2013.0


In [0]:
loans_short=loans[0:1000]

In [75]:
loans_short['disburse_time']=pd.to_datetime(loans_short['disburse_time'])
loans_short['planned_expiration_time']=pd.to_datetime(loans_short['planned_expiration_time'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Metodo poco efficace:

In [0]:
dataframe=pd.DataFrame({'loan_id':[], 'anno':[], 'cash':[]})

#Inizializzo il contatore per le righe del nuovo dataset
k=0

for i in range(0,len(loans_short)):
  
  loan_id=loans_short.iloc[i]['loan_id']
  amount=loans_short.iloc[i]['loan_amount']
  anno_inizio=loans_short.iloc[i]['disburse_time'].year
  anno_fine=loans_short.iloc[i]['planned_expiration_time'].year
  giorni=(loans_short.iloc[i]['planned_expiration_time']-loans_short.iloc[i]['disburse_time']).days
  
  # Non ha senso considerare una planned expiration time precedente alla disburse time
  if giorni >=1:
    #Eseguo il calcolo per i loans aperti e chiusi nello stesso anno
    if anno_inizio==anno_fine:
      dataframe.loc[k]=[loan_id,anno_inizio,amount]
      k+=1

    #Eseguo il calcolo per i loans aperti e chiusi in anni differenti
    else:
      #Creo una lista con tutti gli anni.
      anni=list(range(anno_inizio,anno_fine+1))  
    
      righe=len(anni)
      indice_righe=list(range(k,k+righe+1))

      #Primo anno
      anno=anni[0]
      n=365-pd.to_datetime(loans_short['disburse_time'][i]).dayofyear
      cash=(n*amount)/giorni
      dataframe.loc[indice_righe[0]]=[loan_id,anno,cash]

      #Anni centrali
      for l in range(1,len(anni)-1):
        anno=anni[l]
        cash=(365*amount)/giorni
        dataframe.loc[indice_righe[l]]=[loan_id,anno,cash]

      #Ultimo anno
      anno=anni[-1]
      n=pd.to_datetime(loans_short['planned_expiration_time'][i]).dayofyear
      cash=(n*amount)/giorni
      dataframe.loc[indice_righe[-1]]=[loan_id,anno,cash]

      k=k+righe

In [77]:
dataframe

,loan_id,anno,cash
0,657307.0,2013.0,21.226415
2,657259.0,2013.0,45.833333
4,658010.0,2014.0,400.000000
5,659347.0,2014.0,625.000000
6,656933.0,2013.0,104.385965
...,...,...,...
1010,659317.0,2013.0,22.169811
1012,660410.0,2014.0,125.000000
1013,663077.0,2014.0,225.000000
1014,663849.0,2014.0,1200.000000


In [0]:
new_dataframe=dataframe.groupby('anno')['cash'].sum()

In [79]:
new_dataframe

anno
2012.0    252775.000000
2013.0     10901.806282
2014.0    211475.920748
2015.0    108225.000000
2016.0    125124.902352
2017.0     21025.000000
Name: cash, dtype: float64

Metodo più efficace:

In [0]:
loans['disburse_time']=pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time']=pd.to_datetime(loans['planned_expiration_time'])

In [0]:
dataframe=pd.DataFrame({'anno':[], 'cash':[]})
lista_anni=[]
lista_amount=[]

amounts=[x for x in loans['loan_amount']]
anni_inizio=[x.year for x in loans['disburse_time']]
anni_fine=[x.year for x in loans['planned_expiration_time']]

for i in range(0,len(loans)):
  
  amount=amounts[i]
  anno_inizio=anni_inizio[i]
  anno_fine=anni_fine[i]
  giorni=(loans.iloc[i]['planned_expiration_time']-loans.iloc[i]['disburse_time']).days

  #Non ha senso considerare una planned expiration time precedente alla disburse time
  if giorni >=1:
    #Eseguo il calcolo per i loans aperti e chiusi nello stesso anno
    if anno_inizio==anno_fine:
      lista_anni.append(anno_inizio)
      lista_amount.append(amount)

    #Eseguo il calcolo per i loans aperti e chiusi in anni differenti
    else:
      #Creo una lista con tutti gli anni.
      anni=list(range(anno_inizio,anno_fine+1))  
    
      #Primo anno
      anno=anni[0]
      n=365-pd.to_datetime(loans['disburse_time'][i]).dayofyear
      cash=(n*amount)/giorni
      lista_anni.append(anno)
      lista_amount.append(cash)

      #Anni centrali
      for l in range(1,len(anni)-1):
        anno=anni[l]
        cash=(365*amount)/giorni
        lista_anni.append(anno)
        lista_amount.append(cash)

      #Ultimo anno
      anno=anni[-1]
      n=pd.to_datetime(loans['planned_expiration_time'][i]).dayofyear
      cash=(n*amount)/giorni
      lista_anni.append(anno)
      lista_amount.append(cash)

In [0]:
dataframe=pd.DataFrame({'anno':lista_anni, 'cash':lista_amount})

In [83]:
dataframe

,anno,cash
0,2013,21.226415
1,2014,106.132075
2,2013,45.833333
3,2014,354.166667
4,2014,400.000000
...,...,...
1180842,2016,142.647059
1180843,2015,92.063492
1180844,2016,7.936508
1180845,2015,166.393443


In [0]:
new_dataframe=dataframe.groupby('anno')['cash'].sum()

In [85]:
new_dataframe

anno
2011    6.316783e+05
2012    1.127183e+08
2013    1.235103e+08
2014    1.504951e+08
2015    1.549505e+08
2016    1.532884e+08
2017    1.650673e+08
2018    5.110348e+06
Name: cash, dtype: float64